<a href="https://colab.research.google.com/github/Forpee/NLP-practice/blob/master/text_generation/draft_tf_tut_rap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Text generation

In [9]:
import tensorflow as tf

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-2bb19a8f-a55d-3608-0446-1e466482e094)


In [2]:
!ls '/content/drive/MyDrive/Tf'

 Binance_ICPUSDT_1h.csv   Binance_ICPUSDT_1h.gsheet  'SONG LYRICS.zip'


In [3]:
!unzip '/content/drive/MyDrive/Tf/SONG LYRICS.zip' -d '/content/data'

Archive:  /content/drive/MyDrive/Tf/SONG LYRICS.zip
replace /content/data/Kanye_West.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/data/Kanye_West.txt  
  inflating: /content/data/Lil_Wayne.txt  
  inflating: /content/data/adele.txt  
  inflating: /content/data/al-green.txt  
  inflating: /content/data/alicia-keys.txt  
  inflating: /content/data/amy-winehouse.txt  
  inflating: /content/data/beatles.txt  
  inflating: /content/data/bieber.txt  
  inflating: /content/data/bjork.txt  
  inflating: /content/data/blink-182.txt  
  inflating: /content/data/bob-dylan.txt  
  inflating: /content/data/bob-marley.txt  
  inflating: /content/data/britney-spears.txt  
  inflating: /content/data/bruce-springsteen.txt  
  inflating: /content/data/bruno-mars.txt  
  inflating: /content/data/cake.txt  
  inflating: /content/data/dickinson.txt  
  inflating: /content/data/disney.txt  
  inflating: /content/data/dj-khaled.txt  
  inflating: /content/data/dolly-parton.txt  
  infla

In [4]:
text= open('/content/data/lil-wayne.txt', 'rb').read().decode(encoding='utf-8')
print(f'{len(text)}')

117289


In [6]:
print(text[250:500])

o get a whole key i bought my frist key my first key
Inand we was getting em like for twenty five
Colombian connect homey we was getting fly
We on the grind and our nuts got bigga
And every day we in the motherfucking hood our guns got bigga
Stu


In [7]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

56 unique characters


In [10]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [11]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [12]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [13]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(117289,), dtype=int64, numpy=array([13,  3, 32, ...,  1,  2,  1])>

In [14]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [15]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

I
 
b
o
u
g
h
t
 
m


In [16]:
seq_length = 100

In [17]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'I' b' ' b'b' b'o' b'u' b'g' b'h' b't' b' ' b'm' b'y' b' ' b'f' b'i'
 b'r' b's' b't' b' ' b'k' b'e' b'y' b' ' b'f' b'r' b'o' b'm' b' ' b'm'
 b'y' b' ' b'b' b'a' b'b' b'y' b' ' b'm' b'o' b'm' b'm' b'a' b' ' b'b'
 b'r' b'o' b't' b'h' b'e' b'r' b'\r' b'\n' b'I' b' ' b'b' b'o' b'u' b'g'
 b'h' b't' b' ' b'm' b'y' b' ' b'f' b'i' b'r' b's' b't' b' ' b'k' b'e'
 b'y' b'\r' b'\n' b'B' b'o' b'u' b'g' b'h' b't' b' ' b'm' b'y' b' ' b'b'
 b'o' b'u' b'g' b'h' b't' b' ' b'm' b'y' b' ' b'f' b'i' b'r' b's' b't'
 b' ' b'k' b'e'], shape=(101,), dtype=string)


In [18]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'I bought my first key from my baby momma brother\r\nI bought my first key\r\nBought my bought my first ke'
b'y\r\nI bought my first key from my baby momma brother\r\nI bought my first key\r\nBought my bought my first'
b' key\r\nYeah hustling on my city streets\r\nTrying to get a whole key i bought my frist key my first key\r'
b'\nInand we was getting em like for twenty five\r\nColombian connect homey we was getting fly\r\nWe on the '
b'grind and our nuts got bigga\r\nAnd every day we in the motherfucking hood our guns got bigga\r\nStunting'


In [19]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text
  

In [20]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [21]:
dataset = sequences.map(split_input_target)

In [22]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'I bought my first key from my baby momma brother\r\nI bought my first key\r\nBought my bought my first k'
Target: b' bought my first key from my baby momma brother\r\nI bought my first key\r\nBought my bought my first ke'


In [23]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [24]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [26]:
class MyModel(tf.keras.Model):
  def __init__(self, rnn_units, vocab_size, embedding_dim):
    super().__init__(self)
    self.embedding=tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru=tf.keras.layers.GRU(rnn_units,
                                 return_sequences=True,
                                 return_state=True)
    self.dense=tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x=inputs
    x=self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x=self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x
    

In [27]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)